In [1]:
import math
import pandas as pd 
import numpy as np
import imblearn
from sklearn import preprocessing 
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, classification_report, make_scorer, r2_score, mean_squared_error, explained_variance_score, classification_report
from sklearn.metrics import matthews_corrcoef 
from collections import Counter
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from sklearn.model_selection import  ShuffleSplit, KFold

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier 
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [ ]:
!pip install bootstrapped

  Created wheel for bootstrapped: filename=bootstrapped-0.0.2-py2.py3-none-any.whl size=13954 sha256=b52081aa4b226b406f134edd5504dd542c32fb46e2f1ddfec6549a72591c78f5
  Stored in directory: /root/.cache/pip/wheels/15/55/6a/9a722f067ac4c3dfab359ed2ec7906b9cc6649156d9886bd59
Successfully built bootstrapped


In [2]:
from google.colab import files
uploaded = files.upload()

Saving df.xlsx to df.xlsx


In [3]:
df = pd.read_excel('df.xlsx')

In [4]:
X=df.loc[:,['_1' in i for i in df.columns]]

In [5]:
X = X.drop(["visit:data_visita_1", 'data_prelievo_1','ult_tsa:U_TSA_data_1',  #datetime variables
            "score_recod_1", "ana:età_recod_1",'ana_fis:fumo_recod_1',  #used in the score formula 
      
       'TG_cut_off_1', 'HDL_cut_off_1' , 'glucosio_cut_off_1' , 'pressione_cut_off_1', 'vita_guidelines_cut_off_1'  ,'vita_non_guidelines_cut_off_1' , 'sum_determinanti_guidelines_1',  'sum_determinanti_non_guidelines_1',  'mancanti_guidelines_1', 'mancanti_non_guidelines_1']#other useless variables
           , axis=1) 

In [6]:
y=df['score_recod_4']

In [7]:
n_ext=10

ys = []
for i in range(n_ext):
        ys.append(y)

In [9]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.25, random_state=42)


num_pipeline = Pipeline([
        ('scaler', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('ohe', OneHotEncoder(handle_unknown = 'ignore')),
    ])
    
num_attribs = list( X_train.select_dtypes(include=['int64', 'float64']).columns)
cat_attribs = list( X_train.select_dtypes(include='object').columns)

preprocessor = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

In [16]:
cat_attribs

['ana:istruzione_1',
 'ana_pat:dislipidemia_self_1',
 'ana_pat:tiroide_funzionalità_1']

In [10]:
selector = SelectKBest(score_func=f_regression, k='all')
selector_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('selector', selector)]) 

In [11]:
r2_results=[]

LINEAR SVR 

In [12]:
metrics_df_linear = pd.DataFrame(columns=["Iteration", "Fold", "Features" , "Explained Variance", "MSE", 'R2'])

In [13]:
linear_param = {'model__C': [0.1, 1, 10, 100, 1000] }

linear_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', SVR())]) 






In [23]:

steps=[10, 25, 50, None] #None to have all the features selected 
n_ext=10
feature_ext=[]


for n in range(n_ext):
    k_fold = KFold(5, shuffle=True, random_state=n)
    
    for i, (train_index, test_index) in enumerate(k_fold.split(X, ys[n])):
        X_train_ext, X_val_ext = X.iloc[train_index], X.iloc[test_index]
        y_train_ext, y_val_ext = ys[n][train_index], ys[n][test_index]
        
        tuncv = ShuffleSplit(
                n_splits=5, test_size=0.5, random_state=i
            )
        
        linear_grid = GridSearchCV(estimator = linear_pipe, param_grid = linear_param, cv=tuncv, scoring='r2',n_jobs=-1, verbose=False)

    
    
        linear_grid.fit(X_train_ext,y_train_ext) #GridSearchCV over training  fold to find the optimal parameters 

        best_model = linear_grid.best_estimator_.get_params()['model']
  
        selector_pipe.fit(X_train_ext, y_train_ext) #feature ranking through SelectKBest

        #ordered list of tuples containing the index of the feature and its ranking:
        scores=selector.pvalues_  #or selector.scores_[selector.get_support()]
        feature=list(zip(range(scores.shape[0]), scores))
        feature.sort(key=lambda x:x[1])
        feature_ext.append(feature)

    
        # rescaling
       
     
        X_train_ext = preprocessor.fit_transform(X_train_ext)
        X_val_ext = preprocessor.transform(X_val_ext)
        

        for step in steps:
          selected_features=[tupla[0] for tupla in feature[:step]]
          X_train_fs, X_val_fs = X_train_ext[:, selected_features], X_val_ext[:, selected_features]
          best_model.fit(X_train_fs, y_train)
      
      

          #classif_rep = classification_report(y_ts, yp, output_dict=True)
          y_pred = best_model.predict(X_test_fs)


     


          metrics_df_linear  = metrics_df_linear.append(
                    {
                        "Iteration": n,
                        "Fold": i,
                        "Features": step,
                        "Explained Variance": explained_variance_score(y_val_ext, y_pred),
                        "MSE": mean_squared_error(y_val_ext,y_pred) ,
                      
                          'R2':  r2_score(y_val_ext, y_pred),
                      
                        
                    },
                    ignore_index=True,
                )
          
         
        

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


NameError: ignored

In [ ]:
metrics_df_linear.head()

,Iteration,Fold,Features,Explained Variance,MSE,R2
0,0.0,0.0,10.0,0.077071,22.481826,0.052773
1,0.0,0.0,25.0,0.095494,21.541495,0.092392
2,0.0,0.0,50.0,0.213030,18.684071,0.212784
3,0.0,0.0,NaN,0.209700,18.760897,0.209547
4,0.0,1.0,10.0,0.095298,31.224320,-0.080968


In [ ]:
metrics_df_linear['Features'] = metrics_df_linear['Features'].fillna("all")

In [ ]:
metrics_df_linear.to_csv('metrics_df_linear.csv', encoding = 'utf-8-sig') 

In [ ]:
metrics_df_linear=pd.read_csv('metrics_df_linear.csv')

In [ ]:
avg_df_linear=metrics_df_linear.groupby(['Features']).mean()

In [ ]:
avg_df_linear.to_csv('avg_df_linear.csv', encoding = 'utf-8-sig') 

In [ ]:
avg_df_linear

,Iteration,Fold,Explained Variance,MSE,R2
Features,,,,,
10.0,4.111111,2.0,0.091015,17.685067,0.023946
25.0,4.111111,2.0,0.107731,16.468179,0.070775
50.0,4.111111,2.0,0.164751,15.172782,0.139561
all,4.111111,2.0,0.162344,15.264739,0.137008


In [ ]:
#append confidence intervals
cols=['Explained Variance' ,'MSE', 'R2']

for col in cols:
  avg_df_linear[col]=metrics_df_linear.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).value)
  avg_df_linear['Lower Bound '+ col]=metrics_df_linear.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).lower_bound)
  avg_df_linear['Upper Bound '+ col]=metrics_df_linear.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).upper_bound)

In [ ]:
avg_df_linear.to_csv('CI_df_linear.csv', encoding = 'utf-8-sig') 

In [ ]:
CI_df_linear = pd.read_csv('CI_df_linear.csv')
CI_df_linear

,Features,Unnamed: 0,Iteration,Fold,Explained Variance,MSE,R2,Lower Bound Explained Variance,Upper Bound Explained Variance,Lower Bound MSE,Upper Bound MSE,Lower Bound R2,Upper Bound R2
0,10.0,268,4.111111,2.0,0.091015,17.685067,0.023946,0.078751,0.103063,16.413773,18.899205,0.007940,0.039795
1,25.0,269,4.111111,2.0,0.107731,16.468179,0.070775,0.091889,0.124392,15.448201,17.462298,0.056505,0.085625
2,50.0,270,4.111111,2.0,0.164751,15.172782,0.139561,0.147147,0.183290,14.244184,16.069113,0.122430,0.157481
3,all,271,4.111111,2.0,0.162344,15.264739,0.137008,0.148378,0.177547,14.343151,16.170601,0.122686,0.151514


In [ ]:
r2_results.append (('Linear SVR',
                 CI_df_linear[  (CI_df_linear.R2 == CI_df_linear.R2.values.max())  ] ['R2'].iloc[0],
                 CI_df_linear[  (CI_df_linear.R2 == CI_df_linear.R2.values.max())  ] ['Features'].iloc[0] ))

RADIAL SVR

In [ ]:
metrics_df_radial = pd.DataFrame(columns=["Iteration", "Fold", "Features" , "Explained Variance", "MSE", 'R2'])



In [ ]:
radial_param = [{ 'model__gamma': [ 1e-3, 0.01, 0.1, 0.2, 0.5],'model__C': [1, 10, 100, 1000, 10000]}]

radial_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', SVR(kernel="rbf"))])

In [ ]:
split=0.25  #ok 0.25 is fine 
steps=[10, 25, 50, None] #None to have all the features selected 
n_ext=10

for n in range(n_ext):
    k_fold = KFold(5, shuffle=True, random_state=n)
    
    for i, (train_index, test_index) in enumerate(k_fold.split(X, ys[n])):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_val = ys[n][train_index], ys[n][test_index]
        
        tuncv = ShuffleSplit(
                n_splits=5, test_size=0.5, random_state=i
            )
        
        radial_grid = GridSearchCV(estimator = radial_pipe, param_grid = radial_param, cv=tuncv, scoring='r2',n_jobs=-1, verbose=False)

    
    
        radial_grid.fit(X_train,y_train) #GridSearchCV over training  fold to find the optimal parameters 

        best_model = radial_grid.best_estimator_.get_params()['model']
  
        selector_pipe.fit(X_train, y_train) #feature ranking through SelectKBest

        #ordered list of tuples containing the index of the feature and its ranking:
        scores=selector.pvalues_  #or selector.scores_[selector.get_support()]
        feature=list(zip(range(scores.shape[0]), scores))
        feature.sort(key=lambda x:x[1])

    
        # rescaling
       
     
        X_train = preprocessor.fit_transform(X_train)
        X_test = preprocessor.transform(X_test)
        

        for step in steps:
          selected_features=[tupla[0] for tupla in feature[:step]]
          X_train_fs, X_test_fs = X_train[:, selected_features], X_test[:, selected_features]
          best_model.fit(X_train_fs, y_train)
      
      

          #classif_rep = classification_report(y_ts, yp, output_dict=True)
          y_pred = best_model.predict(X_test_fs)


     


          metrics_df_radial  = metrics_df_radial.append(
                    {
                        "Iteration": n,
                        "Fold": i,
                        "Features": step,
                        "Explained Variance": explained_variance_score(y_val, y_pred),
                        "MSE": mean_squared_error(y_val,y_pred) ,
                      
                          'R2':  r2_score(y_val, y_pred),
                      
                        
                    },
                    ignore_index=True,
                )

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
/usr/local/lib/python3.7/dist-packages/sklearn/feature_

In [ ]:
metrics_df_radial.head()

,Iteration,Fold,Features,Explained Variance,MSE,R2
0,0.0,0.0,10.0,0.014016,17.009709,-0.099771
1,0.0,0.0,25.0,0.167678,14.054959,0.091270
2,0.0,0.0,50.0,0.246368,12.152489,0.214275
3,0.0,0.0,NaN,0.200577,12.713373,0.178011
4,0.0,1.0,10.0,0.290438,11.026956,0.258126


In [ ]:
metrics_df_radial['Features'] = metrics_df_radial['Features'].fillna("all")

In [ ]:
metrics_df_radial.to_csv('metrics_df_radial.csv', encoding = 'utf-8-sig')

In [ ]:
metrics_df_radial=pd.read_csv('metrics_df_radial.csv')

In [ ]:
avg_df_radial=metrics_df_radial.groupby(['Features']).mean()

In [ ]:
avg_df_radial.to_csv('avg_df_radial.csv', encoding = 'utf-8-sig')

In [ ]:
avg_df_radial 

,Iteration,Fold,Explained Variance,MSE,R2
Features,,,,,
10.0,4.5,2.0,0.122216,17.540734,0.054473
25.0,4.5,2.0,0.198843,15.794671,0.145212
50.0,4.5,2.0,0.241660,14.740538,0.202005
all,4.5,2.0,0.229822,14.885957,0.193656


In [ ]:
#append confidence intervals
cols=['Explained Variance' ,'MSE', 'R2']

for col in cols:
  avg_df_radial[col]=metrics_df_radial.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).value)
  avg_df_radial['Lower Bound '+ col]=metrics_df_radial.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).lower_bound)
  avg_df_radial['Upper Bound '+ col]=metrics_df_radial.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).upper_bound)


In [ ]:
avg_df_radial.to_csv('CI_df_radial.csv', encoding = 'utf-8-sig')

In [ ]:
CI_df_radial = pd.read_csv('CI_df_radial.csv')
CI_df_radial 


,Features,Unnamed: 0,Iteration,Fold,Explained Variance,MSE,R2,Lower Bound Explained Variance,Upper Bound Explained Variance,Lower Bound MSE,Upper Bound MSE,Lower Bound R2,Upper Bound R2
0,10.0,98,4.5,2.0,0.122216,17.540734,0.054473,0.093734,0.149942,15.842274,19.173285,0.020132,0.089223
1,25.0,99,4.5,2.0,0.198843,15.794671,0.145212,0.185081,0.212679,14.339539,17.106226,0.128446,0.161651
2,50.0,100,4.5,2.0,0.241660,14.740538,0.202005,0.227989,0.255407,13.379367,16.027373,0.187696,0.216514
3,all,101,4.5,2.0,0.229822,14.885957,0.193656,0.215824,0.244369,13.535836,16.166331,0.179157,0.207930


In [ ]:
r2_results.append (('Radial SVR',
                 CI_df_radial[  (CI_df_radial.R2 == CI_df_radial.R2.values.max())  ] ['R2'].iloc[0],
                 CI_df_radial[  (CI_df_radial.R2 == CI_df_radial.R2.values.max())  ] ['Features'].iloc[0] ))

RANDOM FOREST REGRESSOR

In [ ]:
metrics_df_rf = pd.DataFrame(columns=["Iteration", "Fold", "Features" , "Explained Variance", "MSE", 'R2'])

In [ ]:
rf_param =  {
    
    'model__max_depth': [None, 80, 90,100],
    'model__max_features': ['auto', 'log2' ,2],
    'model__n_estimators': [100, 200, 500]
}

rf_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', RandomForestRegressor())])

In [ ]:

steps=[10, 25, 50, None] #None to have all the features selected 
n_ext=10

for n in range(n_ext):
    k_fold = KFold(5, shuffle=True, random_state=n)
    
    for i, (train_index, test_index) in enumerate(k_fold.split(X, ys[n])):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_val = ys[n][train_index], ys[n][test_index]
        
        tuncv = ShuffleSplit(
                n_splits=5, test_size=0.5, random_state=i
            )
        
        rf_grid = GridSearchCV(estimator = rf_pipe, param_grid = rf_param, cv=tuncv, scoring='r2',n_jobs=-1, verbose=False)

    
    
        rf_grid.fit(X_train,y_train) #GridSearchCV over training  fold to find the optimal parameters 

        best_model = rf_grid.best_estimator_.get_params()['model']
  
        selector_pipe.fit(X_train, y_train) #feature ranking through SelectKBest

        #ordered list of tuples containing the index of the feature and its ranking:
        scores=selector.pvalues_  #or selector.scores_[selector.get_support()]
        feature=list(zip(range(scores.shape[0]), scores))
        feature.sort(key=lambda x:x[1])

    
        # rescaling
       
     
        X_train = preprocessor.fit_transform(X_train)
        X_test = preprocessor.transform(X_test)
        

        for step in steps:
          selected_features=[tupla[0] for tupla in feature[:step]]
          X_train_fs, X_test_fs = X_train[:, selected_features], X_test[:, selected_features]
          best_model.fit(X_train_fs, y_train)
      
      

          #classif_rep = classification_report(y_ts, yp, output_dict=True)
          y_pred = best_model.predict(X_test_fs)


     


          metrics_df_rf  = metrics_df_rf.append(
                    {
                        "Iteration": n,
                        "Fold": i,
                        "Features": step,
                        "Explained Variance": explained_variance_score(y_val, y_pred),
                        "MSE": mean_squared_error(y_val,y_pred) ,
                      
                          'R2':  r2_score(y_val, y_pred),
                      
                        
                    },
                    ignore_index=True,
                )

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
/usr/local/lib/python3.7/dist-packages/sklearn/feature_

KeyboardInterrupt: ignored

In [ ]:
metrics_df_rf.head()

,Iteration,Fold,Features,Explained Variance,MSE,R2
0,0.0,0.0,10.0,-0.003434,15.523764,-0.003697
1,0.0,0.0,25.0,0.064536,14.470454,0.064406
2,0.0,0.0,50.0,0.206725,12.311914,0.203967
3,0.0,0.0,NaN,0.218110,12.130510,0.215696
4,0.0,1.0,10.0,0.231505,11.669857,0.214872


In [ ]:
metrics_df_rf['Features'] = metrics_df_rf['Features'].fillna("all")

In [ ]:
metrics_df_rf.to_csv('metrics_df_rf.csv', encoding = 'utf-8-sig')

In [ ]:
metrics_df_rf=pd.read_csv('metrics_df_rf.csv')

In [ ]:
avg_df_rf=metrics_df_rf.groupby(['Features']).mean()

In [ ]:
avg_df_rf.to_csv('avg_df_xgb.csv', encoding = 'utf-8-sig')
avg_df_rf

,Iteration,Fold,Explained Variance,MSE,R2
Features,,,,,
10.0,1.333333,1.833333,0.076754,17.342969,0.071283
25.0,1.333333,1.833333,0.174204,15.478639,0.169598
50.0,1.333333,1.833333,0.238787,14.291319,0.234803
all,1.333333,1.833333,0.240945,14.253753,0.236795


In [ ]:
#append confidence intervals
cols=['Explained Variance' ,'MSE', 'R2']

for col in cols:
  avg_df_rf[col]=metrics_df_rf.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).value)
  avg_df_rf['Lower Bound '+ col]=metrics_df_rf.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).lower_bound)
  avg_df_rf['Upper Bound '+ col]=metrics_df_rf.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).upper_bound)

In [ ]:
avg_df_rf.to_csv('CI_df_rf.csv', encoding = 'utf-8-sig')

In [ ]:
CI_df_rf = pd.read_csv('CI_df_rf.csv')
CI_df_rf

,Features,Unnamed: 0,Iteration,Fold,Explained Variance,MSE,R2,Lower Bound Explained Variance,Upper Bound Explained Variance,Lower Bound MSE,Upper Bound MSE,Lower Bound R2,Upper Bound R2
0,10.0,20,0.636364,1.818182,0.067020,16.869277,0.063212,0.013444,0.125718,13.883443,19.507169,0.009939,0.122527
1,25.0,21,0.636364,1.818182,0.129323,15.788516,0.125751,0.087707,0.172048,12.962271,18.293053,0.083969,0.168886
2,50.0,22,0.636364,1.818182,0.210825,14.197727,0.208543,0.162414,0.252556,11.852504,16.335986,0.161046,0.249746
3,all,23,0.636364,1.818182,0.227968,13.944195,0.225082,0.180079,0.266630,11.469183,16.130732,0.178349,0.263884


In [ ]:
r2_results.append (('Random Forest',
                 CI_df_rf[  (CI_df_rf.R2 == CI_df_rf.R2.values.max())  ] ['R2'].iloc[0],
                 CI_df_rf[  (CI_df_rf.R2 == CI_df_rf.R2.values.max())  ] ['Features'].iloc[0] ))

In [ ]:
r2_results

[('Random Forest', 0.23679474191220976, 'all')]

XGBoost Regression

In [ ]:
metrics_df_xgb = pd.DataFrame(columns=["Iteration", "Fold", "Features" , "Explained Variance", "MSE", 'R2'])

xgb_param = [ {'model__nthread':[4], #when use hyperthread, xgboost may become slower
              
              'model__learning_rate': [0.03, 0.05, 0.07, 0.1], #so called `eta` value
              'model__max_depth': [5, 6, 7], #importante
          
             
              'model__n_estimators': [100,500,700]}] #importante

xgb_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', XGBRegressor(random_state=42))])


In [ ]:
steps=[10, 25, 50, None] #None to have all the features selected 
n_ext=10


for n in range(n_ext):
    k_fold = KFold(5, shuffle=True, random_state=n)
    
    for i, (train_index, test_index) in enumerate(k_fold.split(X, ys[n])):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_val = ys[n][train_index], ys[n][test_index]
        
        tuncv = ShuffleSplit(
                n_splits=5, test_size=0.5, random_state=i
            )
        
        xgb_grid = GridSearchCV(estimator = xgb_pipe, param_grid = xgb_param, cv=tuncv, scoring='r2',n_jobs=-1, verbose=False)

    
    
        xgb_grid.fit(X_train,y_train) #GridSearchCV over training  fold to find the optimal parameters 

        best_model = xgb_grid.best_estimator_.get_params()['model']
  
        selector_pipe.fit(X_train, y_train) #feature ranking through SelectKBest

        #ordered list of tuples containing the index of the feature and its ranking:
        scores=selector.pvalues_  #or selector.scores_[selector.get_support()]
        feature=list(zip(range(scores.shape[0]), scores))
        feature.sort(key=lambda x:x[1])

    
        # rescaling
       
     
        X_train = preprocessor.fit_transform(X_train)
        X_test = preprocessor.transform(X_test)
        

        for step in steps:
          selected_features=[tupla[0] for tupla in feature[:step]]
          X_train_fs, X_test_fs = X_train[:, selected_features], X_test[:, selected_features]
          best_model.fit(X_train_fs, y_train)
      
      

          #classif_rep = classification_report(y_ts, yp, output_dict=True)
          y_pred = best_model.predict(X_test_fs)


     


          metrics_df_xgb  = metrics_df_xgb.append(
                    {
                        "Iteration": n,
                        "Fold": i,
                        "Features": step,
                        "Explained Variance": explained_variance_score(y_val, y_pred),
                        "MSE": mean_squared_error(y_val,y_pred) ,
                      
                          'R2':  r2_score(y_val, y_pred),
                      
                        
                    },
                    ignore_index=True,
                )

[14:39:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


[14:39:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:39:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:39:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:39:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:43:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


[14:43:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:43:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:43:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:43:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:46:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


...

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


[15:10:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:10:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:10:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:10:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:13:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms


[15:13:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:13:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:13:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:13:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


KeyboardInterrupt: ignored

In [ ]:
metrics_df_xgb.head()

,Iteration,Fold,Features,Explained Variance,MSE,R2
0,0.0,0.0,10.0,0.048333,14.747911,0.046467
1,0.0,0.0,25.0,0.015747,15.285035,0.011739
2,0.0,0.0,50.0,0.139140,13.321942,0.138663
3,0.0,0.0,NaN,0.172302,12.812741,0.171586
4,0.0,1.0,10.0,0.196503,11.965474,0.194984


In [ ]:
metrics_df_xgb['Features'] = metrics_df_xgb['Features'].fillna("all")

In [ ]:
metrics_df_xgb.to_csv('metrics_df_xgb.csv', encoding = 'utf-8-sig')

In [ ]:
avg_df_xgb=metrics_df_xgb.groupby(['Features']).mean()

In [ ]:
avg_df_xgb.to_csv('avg_df_xgb.csv', encoding = 'utf-8-sig')

avg_df_xgb


,Iteration,Fold,Explained Variance,MSE,R2
Features,,,,,
10.0,0.636364,1.818182,0.067020,16.869277,0.063212
25.0,0.636364,1.818182,0.129323,15.788516,0.125751
50.0,0.636364,1.818182,0.210825,14.197727,0.208543
all,0.636364,1.818182,0.227968,13.944195,0.225082


In [ ]:
#append confidence intervals
cols=['Explained Variance' ,'MSE', 'R2']

for col in cols:
  avg_df_xgb[col]=metrics_df_xgb.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).value)
  avg_df_xgb['Lower Bound '+ col]=metrics_df_xgb.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).lower_bound)
  avg_df_xgb['Upper Bound '+ col]=metrics_df_xgb.groupby('Features')[col].apply(lambda x:bs.bootstrap(x.values ,stat_func=bs_stats.mean ).upper_bound)

In [ ]:
avg_df_xgb.to_csv('CI_df_xgb.csv', encoding = 'utf-8-sig')

In [ ]:

CI_df_xgb = pd.read_csv('CI_df_xgb.csv')
CI_df_xgb

,Features,Unnamed: 0,Iteration,Fold,Explained Variance,MSE,R2,Lower Bound Explained Variance,Upper Bound Explained Variance,Lower Bound MSE,Upper Bound MSE,Lower Bound R2,Upper Bound R2
0,10.0,20,0.636364,1.818182,0.067020,16.869277,0.063212,0.013444,0.125718,13.883443,19.507169,0.009939,0.122527
1,25.0,21,0.636364,1.818182,0.129323,15.788516,0.125751,0.087707,0.172048,12.962271,18.293053,0.083969,0.168886
2,50.0,22,0.636364,1.818182,0.210825,14.197727,0.208543,0.162414,0.252556,11.852504,16.335986,0.161046,0.249746
3,all,23,0.636364,1.818182,0.227968,13.944195,0.225082,0.180079,0.266630,11.469183,16.130732,0.178349,0.263884


In [ ]:
r2_results.append (('XGBoost Regression',
                 CI_df_xgb[  (CI_df_xgb.R2 == CI_df_xgb.R2.values.max())  ] ['R2'].iloc[0],
                 CI_df_xgb[  (CI_df_xgb.R2 == CI_df_xgb.R2.values.max())  ] ['Features'].iloc[0] ))

CONCLUSIONS

In [ ]:
r2_results

[('Random Forest', 0.23679474191220976, 'all'),
 ('XGBoost Regression', 0.2250824085072309, 'all'),
 ('Linear SVR', 0.13956117089811007, '50.0'),
 ('Radial SVR', 0.20200460470018824, '50.0')]